In [1]:
pip install pandas

In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [4]:
df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,2024-09-04 13:36:23,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,2024-09-04 14:35:32,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,2024-09-05 13:50:59,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,2024-09-05 16:08:47,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [23]:
from datetime import datetime, timedelta

# Define the start and end dates
start_date = datetime(2019, 7, 1)
end_date = datetime(2024, 10, 1)

# Initialize an empty list to store the timestamps
timestamps = []

# Loop through each month from start_date to end_date
current_date = start_date
while current_date <= end_date:
    # Append the current date to the list
    timestamps.append(current_date)
    # Move to the first day of the next month
    next_month = current_date.month % 12 + 1
    next_year = current_date.year + (current_date.month // 12)
    current_date = datetime(next_year, next_month, 1)

len(timestamps), timestamps[0], timestamps[-1]

(64, datetime.datetime(2019, 7, 1, 0, 0), datetime.datetime(2024, 10, 1, 0, 0))

In [24]:
waterfall_duration = [] # Sum of activity every 7 days
waterfall_x = [] # Timestamp
waterfall_y = [] # Change of activity every 7 days
waterfall_text = [] # Change of activity as string text
activity = 0

for i in range(len(timestamps)-1):
  start_month = timestamps[i]
  end_month = timestamps[i+1]

  duration = df0[(df0['timestamp'] >= str(start_month)) & (df0['timestamp'] <= str(end_month))]['duration'].sum()
  duration = int(duration)
  y = duration - activity
  text = '+' + str(y) if y >= 0 else str(y)

  waterfall_duration.append(duration)
  waterfall_x.append(start_month)
  waterfall_y.append(y)
  waterfall_text.append(text)

  activity = duration

In [25]:
waterfall_n_weeks = [1]
waterfall_measure = ['total']

for i, timestamp in enumerate(waterfall_x[1:-1]):
  waterfall_n_weeks.append(i+1)
  if timestamp.month == 1:
    waterfall_measure.append('total')
  else:
    waterfall_measure.append('relative')

waterfall_n_weeks.append(len(waterfall_duration))
waterfall_measure.append("total")

len(waterfall_n_weeks), len(waterfall_measure)

(63, 63)

In [26]:
waterfall_df = pd.DataFrame({
    'duration': waterfall_duration,
    'x': waterfall_x,
    'y': waterfall_y,
    'text': waterfall_text,
    'n_weeks': waterfall_n_weeks,
    'measure': waterfall_measure
})

waterfall_df

,duration,x,y,text,n_weeks,measure
0,2619,2019-07-01,2619,+2619,1,total
1,1944,2019-08-01,-675,-675,1,relative
2,1408,2019-09-01,-536,-536,2,relative
3,1310,2019-10-01,-98,-98,3,relative
4,2467,2019-11-01,1157,+1157,4,relative
...,...,...,...,...,...,...
58,9611,2024-05-01,4003,+4003,58,relative
59,7072,2024-06-01,-2539,-2539,59,relative
60,2278,2024-07-01,-4794,-4794,60,relative
61,4978,2024-08-01,2700,+2700,61,relative


In [27]:
waterfall_df['measure'].value_counts()

,count
measure,
relative,56
total,7


In [28]:
waterfall_df['duration'].sum(), df0['duration'].sum()

(163424, 163455.15999999997)

In [30]:
waterfall_df.to_csv('8_waterfall.csv', index=False)